In [29]:

def input_mlflow(cv_r_v,increment_number,X_train, y_train,pipeline,run_name,experiment_id,X_test,y_test):
    

    for wi in range(len(cv_r_v)):
        
        #====================skema===================
        cols_spec = []
        data_map = {
                'int64': 'integer',
                'float64': 'double',
                'bool': 'boolean',
                'str': 'string',
                "date": 'datetime'
            }

        for name, dtype in X_train.dtypes.to_dict().items():
            cols_spec.append(ColSpec(name=name, type=data_map[str(dtype)]))
        input_schema = Schema(inputs=cols_spec)
        output_schema = Schema([ColSpec(name="label", type="double")])
        #parameter = ParamSpec(name="model_name", dtype="string", default="model1")
        #param_schema = ParamSchema(params=[parameter])
        model_signature = ModelSignature(inputs=input_schema, outputs=output_schema)#, params=param_schema)
        #print("MODEL SIGNATURE")
        #print(model_signature.to_dict())

        model_signature = infer_signature(X_train, y_train)#, params={"model_name": "model1"})
        #print("MODEL SIGNATURE")
        #print(model_signature.to_dict())

        
        
        
        
        #====================buat run baru===================
        run_name_with_increment = f"{run_name}__{increment_number}"
            # Membuka run MLflow
        with mlflow.start_run(run_name=run_name_with_increment, experiment_id=experiment_id) as run:
            # Mendapatkan run_id
            run_id = mlflow.active_run().info.run_id
        print(f"'mlruns/{experiment_id}/{run_id}/artifacts/grid_search__{increment_number}',")
        increment_number=increment_number+1

        
        
        #====================buat parameter model dan metric===================

        # Definisikan grid parameter untuk dicari
        param_grid = cv_r_v[wi]
        
        #====================model===================

        # Inisialisasi GridSearchCV
        grid_search = GridSearchCV(pipeline, param_grid, cv=5,  scoring='neg_mean_squared_error')

        # Lakukan pencarian grid
        grid_search.fit(X_train, y_train)
        
            
        #====================parameter===================
        pipe=grid_search.best_estimator_
        
        
        first_step_name = list(pipe.named_steps.keys())[0:len(pipe)]
        

        for i in range(len(pipe)):
            # Mendapatkan parameter dari langkah 'sca'
            nama=first_step_name[i]
            sca_params = pipe.get_params()[nama]

                # Membuka run MLflow
            with mlflow.start_run( experiment_id=experiment_id,run_id=run_id) as run:

                # Log parameter secara otomatis menggunakan loop
                for param_name, param_value in sca_params.get_params().items():
                    param_name=nama+'__'+param_name
                    #print(param_name,param_value)
                    mlflow.log_param(param_name, param_value)
                    
                    
                #====================metric===================
                #matric
                hasil_test=grid_search.predict(X_test)
                mae=mean_absolute_error(y_test, hasil_test)
                mse=mean_squared_error(y_test, hasil_test)
                rmse=np.sqrt(mse)
                r2=r2_score(y_test, hasil_test)
                #print(mse)
                

                # log model 
                mlflow.sklearn.log_model(sk_model=grid_search, artifact_path="grid_search__"+str(increment_number-1),signature=model_signature)

                metrics = {
                    'mean_test_score':pd.DataFrame(grid_search.cv_results_)['mean_test_score'].values[0],
                    "mse": mse,
                    "mae": mae,
                    "rmse": rmse,
                    "r2": r2
                }

                mlflow.log_metrics(metrics)
                
                
        mlflow.end_run()
    print('selesai')

        
    return increment_number


            
        

def ambil_best(grid_search,n):
    cv_r=grid_search.copy()
    cv_r_v=cv_r.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
    cv_r_v=cv_r_v.head(n)['params'].values

    for wi in range(len(cv_r_v)):

        for i in list(cv_r_v[wi]):

            cv_r_v[wi][i]=[cv_r_v[wi][i]]


    return cv_r_v


In [2]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats



import seaborn as sns
import statsmodels.api as sm
import pandas as pd



from scipy.stats import shapiro,normaltest,kstest,jarque_bera


import pingouin as pg
from pingouin import kruskal


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import mlflow


from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score


import mlflow

from mlflow.models.signature import ModelSignature
from mlflow.models.signature import infer_signature
from mlflow.types.schema import Schema
from mlflow.types.schema import ParamSchema
from mlflow.types.schema import ParamSpec
from mlflow.types.schema import ColSpec
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import pandas as pd
from typing import Tuple
from sklearn.svm import SVR,LinearSVR

from sklearn.feature_selection import VarianceThreshold

In [4]:
df = pd.read_csv('data/data_csv/data_bersih.csv')

In [ ]:
SVMRegresion

,Cement,Slag,Fly ash,Water,SP,Fine Aggr.,SLUMP(cm),FLOW(cm),Compressive Strength (28-day)(Mpa)
0,273.0,82.0,105.0,210.0,9.0,680.0,23.0,62.0,34.99
1,163.0,149.0,191.0,180.0,12.0,746.0,0.0,20.0,41.14
2,162.0,148.0,191.0,179.0,16.0,743.0,1.0,20.0,41.81
3,154.0,112.0,144.0,220.0,10.0,658.0,20.0,64.0,26.82
4,147.0,89.0,115.0,202.0,9.0,829.0,23.0,55.0,25.21


In [7]:
df.columns=['Cement', 'Slag', 'Fly ash', 'Water', 'SP',
       'Fine Aggr.', 'SLUMP(cm)', 'FLOW(cm)',
       'CS(Mpa)']

In [8]:
X = df.drop(['CS(Mpa)'],axis=1)
y = df['CS(Mpa)']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=101)

In [70]:
from sklearn.feature_selection import mutual_info_regression
# determine the mutual information
mutual_info = mutual_info_regression(X_train.fillna(0), y_train)
mutual_info

mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

#newspaper kemungkinan drop

Fly ash       0.375505
Cement        0.322531
Slag          0.251079
Water         0.181348
SLUMP(cm)     0.157396
SP            0.145470
FLOW(cm)      0.135938
Fine Aggr.    0.045462
dtype: float64

In [10]:
from sklearn impor  t set_config
set_config(display='diagram')

In [11]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

svmr = SVR()
scatler=StandardScaler()

opera1=[('StandardScaler',scatler),('SVR',svmr)]
pipeline1=Pipeline(opera1)
pipeline1

Pipeline(steps=[('StandardScaler', StandardScaler()), ('SVR', SVR())])

In [33]:
kelip=[0.001,0.01,0.1,1,10]

param_grid = {
    'SVR__kernel': ['linear', 'rbf'],        # Derajat polinomial yang akan dicoba
    
    'SVR__gamma':['scale', 'auto'],
    'SVR__C':kelip,
    'SVR__epsilon':kelip
}

In [34]:


from sklearn.model_selection import GridSearchCV
# Inisialisasi GridSearchCV


# Inisialisasi GridSearchCV dengan menggunakan MAE sebagai skor evaluasi
grid_search = GridSearchCV(pipeline1, param_grid, cv=5,  scoring='neg_mean_squared_error')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('SVR', SVR())]),
             param_grid={'SVR__C': [0.001, 0.01, 0.1, 1, 10],
                         'SVR__epsilon': [0.001, 0.01, 0.1, 1, 10],
                         'SVR__gamma': ['scale', 'auto'],
                         'SVR__kernel': ['linear', 'rbf']},
             scoring='neg_mean_squared_error')

In [35]:
df_1 = pd.DataFrame(grid_search.cv_results_)
df_1.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_1 = df_1.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_SVR__C,param_SVR__epsilon,param_SVR__gamma,param_SVR__kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
80,0.014226,0.003910,0.001337,0.002675,10,0.001,scale,linear,"{'SVR__C': 10, 'SVR__epsilon': 0.001, 'SVR__ga...",-11.260417,-4.118103,-8.653493,-4.855549,-8.424334,-7.462379,2.636246,1
84,0.014085,0.000829,0.004086,0.002595,10,0.01,scale,linear,"{'SVR__C': 10, 'SVR__epsilon': 0.01, 'SVR__gam...",-11.285324,-4.122717,-8.634854,-4.885701,-8.415008,-7.468721,2.633990,3
88,0.009128,0.003402,0.001939,0.002771,10,0.1,scale,linear,"{'SVR__C': 10, 'SVR__epsilon': 0.1, 'SVR__gamm...",-11.409928,-4.216861,-8.818935,-5.293357,-8.398450,-7.627506,2.584621,5
92,0.012039,0.003267,0.004673,0.003049,10,1,scale,linear,"{'SVR__C': 10, 'SVR__epsilon': 1, 'SVR__gamma'...",-12.991486,-4.805994,-8.746730,-5.779466,-7.918349,-8.048405,2.849559,7
64,0.009156,0.001546,0.001847,0.002331,1,0.01,scale,linear,"{'SVR__C': 1, 'SVR__epsilon': 0.01, 'SVR__gamm...",-16.643701,-4.662861,-10.418571,-4.693167,-5.824599,-8.448580,4.612179,9
60,0.006892,0.001534,0.002226,0.002710,1,0.001,scale,linear,"{'SVR__C': 1, 'SVR__epsilon': 0.001, 'SVR__gam...",-16.720795,-4.656389,-10.413708,-4.686974,-5.804332,-8.456440,4.643563,11
72,0.008172,0.001689,0.001570,0.001923,1,1,scale,linear,"{'SVR__C': 1, 'SVR__epsilon': 1, 'SVR__gamma':...",-15.174956,-5.987259,-10.594981,-5.363948,-5.196287,-8.463486,3.898731,13
68,0.008192,0.005068,0.001202,0.001276,1,0.1,scale,linear,"{'SVR__C': 1, 'SVR__epsilon': 0.1, 'SVR__gamma...",-16.845687,-4.711412,-10.479784,-4.760899,-5.834535,-8.526464,4.669165,15
89,0.008477,0.003642,0.001958,0.002449,10,0.1,scale,rbf,"{'SVR__C': 10, 'SVR__epsilon': 0.1, 'SVR__gamm...",-30.162613,-9.355873,-5.949068,-5.467978,-6.098048,-11.406716,9.478672,17
85,0.009746,0.003563,0.001420,0.001760,10,0.01,scale,rbf,"{'SVR__C': 10, 'SVR__epsilon': 0.01, 'SVR__gam...",-30.234503,-9.464000,-5.939554,-5.580407,-6.435624,-11.530818,9.452303,19


In [36]:
param_grid2 = {
    'SVR__kernel': ['sigmoid'],        # Derajat polinomial yang akan dicoba
    
    'SVR__gamma':['scale', 'auto'],
    'SVR__C':kelip,
    'SVR__epsilon':kelip,
    'SVR__coef0':kelip
}

In [37]:


from sklearn.model_selection import GridSearchCV
# Inisialisasi GridSearchCV


# Inisialisasi GridSearchCV dengan menggunakan MAE sebagai skor evaluasi
grid_search = GridSearchCV(pipeline1, param_grid2, cv=5,  scoring='neg_mean_squared_error')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('SVR', SVR())]),
             param_grid={'SVR__C': [0.001, 0.01, 0.1, 1, 10],
                         'SVR__coef0': [0.001, 0.01, 0.1, 1, 10],
                         'SVR__epsilon': [0.001, 0.01, 0.1, 1, 10],
                         'SVR__gamma': ['scale', 'auto'],
                         'SVR__kernel': ['sigmoid']},
             scoring='neg_mean_squared_error')

In [38]:
df_2 = pd.DataFrame(grid_search.cv_results_)
df_2.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_2 = df_2.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_SVR__C,param_SVR__coef0,param_SVR__epsilon,param_SVR__gamma,param_SVR__kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
156,0.008040,0.003738,0.006452,0.003151,1,0.001,1,scale,sigmoid,"{'SVR__C': 1, 'SVR__coef0': 0.001, 'SVR__epsil...",-59.727682,-35.340849,-30.052890,-32.265016,-15.262956,-34.529879,14.366449,1
166,0.011933,0.004154,0.000490,0.000980,1,0.01,1,scale,sigmoid,"{'SVR__C': 1, 'SVR__coef0': 0.01, 'SVR__epsilo...",-59.845366,-35.629350,-30.183254,-32.339986,-15.337490,-34.667089,14.380717,3
154,0.009309,0.002411,0.002285,0.001470,1,0.001,0.1,scale,sigmoid,"{'SVR__C': 1, 'SVR__coef0': 0.001, 'SVR__epsil...",-61.515845,-37.645677,-32.455308,-34.636462,-14.834248,-36.217508,14.939499,5
228,0.005636,0.003052,0.001047,0.001163,10,0.1,10,scale,sigmoid,"{'SVR__C': 10, 'SVR__coef0': 0.1, 'SVR__epsilo...",-50.220782,-45.840636,-25.765147,-28.550152,-30.945259,-36.264395,9.844004,7
152,0.003372,0.004132,0.003258,0.005450,1,0.001,0.01,scale,sigmoid,"{'SVR__C': 1, 'SVR__coef0': 0.001, 'SVR__epsil...",-61.666501,-38.377720,-32.618106,-34.484573,-14.875835,-36.404547,14.990454,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.005507,0.003045,0.002636,0.002790,10,1,10,scale,sigmoid,"{'SVR__C': 10, 'SVR__coef0': 1, 'SVR__epsilon'...",-233.709336,-55.267988,-63.606172,-76.957142,-42.181745,-94.344477,70.594120,241
234,0.006025,0.002608,0.003949,0.002464,10,1,0.1,scale,sigmoid,"{'SVR__C': 10, 'SVR__coef0': 1, 'SVR__epsilon'...",-407.174362,-155.353959,-256.086584,-147.339758,-153.118869,-223.814706,100.195240,243
232,0.007022,0.003256,0.003766,0.003020,10,1,0.01,scale,sigmoid,"{'SVR__C': 10, 'SVR__coef0': 1, 'SVR__epsilon'...",-418.719346,-156.113435,-287.130479,-144.652852,-153.808886,-232.085000,107.149982,245
230,0.007324,0.005728,0.002903,0.005083,10,1,0.001,scale,sigmoid,"{'SVR__C': 10, 'SVR__coef0': 1, 'SVR__epsilon'...",-418.845256,-156.190304,-287.255752,-144.389618,-153.915795,-232.119345,107.223233,247


In [39]:
param_grid3 = {
    'SVR__kernel': ['poly'],
    'SVR__degree': [2,3,4,5,6,7,8],# Derajat polinomial yang akan dicoba
    
    'SVR__gamma':['scale', 'auto'],
    'SVR__C':kelip,
    'SVR__epsilon':kelip,
    'SVR__coef0':kelip
}

In [40]:


from sklearn.model_selection import GridSearchCV
# Inisialisasi GridSearchCV


# Inisialisasi GridSearchCV dengan menggunakan MAE sebagai skor evaluasi
grid_search = GridSearchCV(pipeline1, param_grid3, cv=5,  scoring='neg_mean_squared_error')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('SVR', SVR())]),
             param_grid={'SVR__C': [0.001, 0.01, 0.1, 1, 10],
                         'SVR__coef0': [0.001, 0.01, 0.1, 1, 10],
                         'SVR__degree': [2, 3, 4, 5, 6, 7, 8],
                         'SVR__epsilon': [0.001, 0.01, 0.1, 1, 10],
                         'SVR__gamma': ['scale', 'auto'],
                         'SVR__kernel': ['poly']},
             scoring='neg_mean_squared_error')

In [64]:


from sklearn.model_selection import GridSearchCV
# Inisialisasi GridSearchCV


# Inisialisasi GridSearchCV dengan menggunakan MAE sebagai skor evaluasi
grid_search = GridSearchCV(pipeline1, param_grid3, cv=5,  scoring='neg_mean_squared_error')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('SVR', SVR())]),
             param_grid={'SVR__C': [0.001, 0.01, 0.1, 1, 10],
                         'SVR__coef0': [0.001, 0.01, 0.1, 1, 10],
                         'SVR__degree': [2, 3, 4, 5, 6, 7, 8],
                         'SVR__epsilon': [0.001, 0.01, 0.1, 1, 10],
                         'SVR__gamma': ['scale', 'auto'],
                         'SVR__kernel': ['poly']},
             scoring='neg_mean_squared_error')

In [66]:
df_3 = pd.DataFrame(grid_search.cv_results_)
df_3.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_3 = df_3.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_3

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_SVR__C,param_SVR__coef0,param_SVR__degree,param_SVR__epsilon,param_SVR__gamma,param_SVR__kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1342,0.085239,0.016543,0.000000,0.000000,1,10,3,0.01,scale,poly,"{'SVR__C': 1, 'SVR__coef0': 10, 'SVR__degree':...",-1.170912,-0.519008,-0.512223,-0.131525,-0.235938,-0.513921,0.361986,1
1340,0.076565,0.022778,0.002344,0.004689,1,10,3,0.001,scale,poly,"{'SVR__C': 1, 'SVR__coef0': 10, 'SVR__degree':...",-1.170652,-0.545111,-0.515115,-0.134314,-0.239582,-0.520955,0.360939,3
1344,0.072564,0.021829,0.003963,0.006055,1,10,3,0.1,scale,poly,"{'SVR__C': 1, 'SVR__coef0': 10, 'SVR__degree':...",-1.393044,-0.331110,-0.502630,-0.171620,-0.256879,-0.531056,0.444562,5
1684,0.062641,0.012185,0.005168,0.005712,10,10,2,0.1,scale,poly,"{'SVR__C': 10, 'SVR__coef0': 10, 'SVR__degree'...",-0.970662,-0.546261,-0.804726,-0.744546,-0.536191,-0.720477,0.164063,7
1682,0.071329,0.025386,0.004649,0.005656,10,10,2,0.01,scale,poly,"{'SVR__C': 10, 'SVR__coef0': 10, 'SVR__degree'...",-1.187944,-0.399225,-0.854432,-0.693999,-0.519973,-0.731115,0.275684,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402,0.004293,0.005287,0.006185,0.006611,10,0.001,2,0.01,scale,poly,"{'SVR__C': 10, 'SVR__coef0': 0.001, 'SVR__degr...",-110.320169,-76.080142,-46.729003,-35.390697,-63.711261,-66.446254,25.993961,1741
1400,0.007982,0.007008,0.001313,0.002626,10,0.001,2,0.001,scale,poly,"{'SVR__C': 10, 'SVR__coef0': 0.001, 'SVR__degr...",-110.303269,-76.059255,-46.726076,-35.454169,-63.973530,-66.503260,25.966673,1743
1462,0.014812,0.001644,0.000000,0.000000,10,0.001,8,0.01,scale,poly,"{'SVR__C': 10, 'SVR__coef0': 0.001, 'SVR__degr...",-136.525178,-42.808442,-49.204060,-41.533300,-63.200141,-66.654224,35.771696,1745
1460,0.006645,0.008161,0.006052,0.007420,10,0.001,8,0.001,scale,poly,"{'SVR__C': 10, 'SVR__coef0': 0.001, 'SVR__degr...",-136.562941,-42.812656,-49.192942,-41.520963,-63.583842,-66.734669,35.781619,1747


In [67]:
df_gabungan=pd.concat([df_1.head(12),df_2.head(7),df_3.head(12)])
df_gabungan.drop_duplicates(subset=['mean_test_score', 'std_test_score'], inplace=True)
df_gabungan.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_SVR__C,param_SVR__epsilon,param_SVR__gamma,param_SVR__kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,param_SVR__coef0,param_SVR__degree
1342,0.085239,0.016543,0.000000,0.000000,1,0.01,scale,poly,"{'SVR__C': 1, 'SVR__coef0': 10, 'SVR__degree':...",-1.170912,-0.519008,-0.512223,-0.131525,-0.235938,-0.513921,0.361986,1,10,3
1340,0.076565,0.022778,0.002344,0.004689,1,0.001,scale,poly,"{'SVR__C': 1, 'SVR__coef0': 10, 'SVR__degree':...",-1.170652,-0.545111,-0.515115,-0.134314,-0.239582,-0.520955,0.360939,3,10,3
1344,0.072564,0.021829,0.003963,0.006055,1,0.1,scale,poly,"{'SVR__C': 1, 'SVR__coef0': 10, 'SVR__degree':...",-1.393044,-0.331110,-0.502630,-0.171620,-0.256879,-0.531056,0.444562,5,10,3
1684,0.062641,0.012185,0.005168,0.005712,10,0.1,scale,poly,"{'SVR__C': 10, 'SVR__coef0': 10, 'SVR__degree'...",-0.970662,-0.546261,-0.804726,-0.744546,-0.536191,-0.720477,0.164063,7,10,2
1682,0.071329,0.025386,0.004649,0.005656,10,0.01,scale,poly,"{'SVR__C': 10, 'SVR__coef0': 10, 'SVR__degree'...",-1.187944,-0.399225,-0.854432,-0.693999,-0.519973,-0.731115,0.275684,9,10,2
1680,0.079733,0.041478,0.006255,0.005905,10,0.001,scale,poly,"{'SVR__C': 10, 'SVR__coef0': 10, 'SVR__degree'...",-1.243954,-0.390862,-0.857250,-0.698444,-0.529834,-0.744069,0.295103,11,10,2
334,0.014489,0.004366,0.003132,0.006264,0.001,0.1,scale,poly,"{'SVR__C': 0.001, 'SVR__coef0': 10, 'SVR__degr...",-1.854305,-0.380177,-0.995007,-0.232452,-0.264696,-0.745328,0.619605,13,10,7
324,0.028812,0.004707,0.006968,0.006403,0.001,0.1,scale,poly,"{'SVR__C': 0.001, 'SVR__coef0': 10, 'SVR__degr...",-1.840453,-0.355408,-1.099981,-0.214005,-0.234305,-0.748830,0.635689,23,10,6
312,0.021107,0.007396,0.004046,0.006078,0.001,0.01,scale,poly,"{'SVR__C': 0.001, 'SVR__coef0': 10, 'SVR__degr...",-1.379773,-0.380188,-0.953183,-0.578244,-0.520204,-0.762319,0.362272,33,10,5
310,0.025206,0.007653,0.005767,0.006085,0.001,0.001,scale,poly,"{'SVR__C': 0.001, 'SVR__coef0': 10, 'SVR__degr...",-1.380916,-0.388989,-0.950942,-0.580623,-0.522180,-0.764730,0.360074,35,10,5


In [68]:
inis2=ambil_best(df_gabungan,10)
inis2

array([{'SVR__C': [1], 'SVR__coef0': [10], 'SVR__degree': [3], 'SVR__epsilon': [0.01], 'SVR__gamma': ['scale'], 'SVR__kernel': ['poly']},
       {'SVR__C': [1], 'SVR__coef0': [10], 'SVR__degree': [3], 'SVR__epsilon': [0.001], 'SVR__gamma': ['scale'], 'SVR__kernel': ['poly']},
       {'SVR__C': [1], 'SVR__coef0': [10], 'SVR__degree': [3], 'SVR__epsilon': [0.1], 'SVR__gamma': ['scale'], 'SVR__kernel': ['poly']},
       {'SVR__C': [10], 'SVR__coef0': [10], 'SVR__degree': [2], 'SVR__epsilon': [0.1], 'SVR__gamma': ['scale'], 'SVR__kernel': ['poly']},
       {'SVR__C': [10], 'SVR__coef0': [10], 'SVR__degree': [2], 'SVR__epsilon': [0.01], 'SVR__gamma': ['scale'], 'SVR__kernel': ['poly']},
       {'SVR__C': [10], 'SVR__coef0': [10], 'SVR__degree': [2], 'SVR__epsilon': [0.001], 'SVR__gamma': ['scale'], 'SVR__kernel': ['poly']},
       {'SVR__C': [0.001], 'SVR__coef0': [10], 'SVR__degree': [7], 'SVR__epsilon': [0.1], 'SVR__gamma': ['scale'], 'SVR__kernel': ['poly']},
       {'SVR__C': [0.001], '

In [44]:
increment_number=input_mlflow(inis2,increment_number,X_train, y_train,pipeline1,run_name,experiment_id,X_test,y_test)

'mlruns/721665528993214933/0a9bb5706ecf4d869ffdb50cf85781cb/artifacts/grid_search__1',
'mlruns/721665528993214933/a2a1fba817a940bba070b0bd5ddb679a/artifacts/grid_search__2',
'mlruns/721665528993214933/a7f1448c63c94957a95be7c0cae40b97/artifacts/grid_search__3',
'mlruns/721665528993214933/06c08794130b412280899bd077b69ea5/artifacts/grid_search__4',
'mlruns/721665528993214933/6bf1fb6c53b548f093473171f9407fc7/artifacts/grid_search__5',
'mlruns/721665528993214933/574361b777244a81b66a74631fb29148/artifacts/grid_search__6',
'mlruns/721665528993214933/3aa73043032f48e086154fc53eb7859a/artifacts/grid_search__7',
'mlruns/721665528993214933/853a241fffc64c8abe9ab97875bfda0f/artifacts/grid_search__8',
'mlruns/721665528993214933/05000b3ef3d740389c5232617f8066cf/artifacts/grid_search__9',
'mlruns/721665528993214933/39f6dcec9dfd47478762d54d65122ee0/artifacts/grid_search__10',
selesai


In [45]:

def test_reg(model_paths,X_test2,y_test):
    ii=1
    for i, model_path in enumerate(model_paths, start=1):
        print(f'\n\n================================| model {ii} |==========================================')
        
        model = mlflow.sklearn.load_model(model_path)
        hasil_test = model.predict(X_test2)
        
        mae=mean_absolute_error(y_test, hasil_test)
        mse=mean_squared_error(y_test, hasil_test)
        rmse=np.sqrt(mse)
        r2=r2_score(y_test, hasil_test)
        
        print(f'mae = {mae}\nmse = {mse}\nrmse = {rmse}\nr2 = {r2}')
        
        ii=ii+1

In [47]:
from sklearn.metrics import classification_report, confusion_matrix
model_paths = [
    'mlruns/721665528993214933/0a9bb5706ecf4d869ffdb50cf85781cb/artifacts/grid_search__1',
    'mlruns/721665528993214933/a2a1fba817a940bba070b0bd5ddb679a/artifacts/grid_search__2',
    'mlruns/721665528993214933/a7f1448c63c94957a95be7c0cae40b97/artifacts/grid_search__3',
    'mlruns/721665528993214933/06c08794130b412280899bd077b69ea5/artifacts/grid_search__4',
    'mlruns/721665528993214933/6bf1fb6c53b548f093473171f9407fc7/artifacts/grid_search__5',
    'mlruns/721665528993214933/574361b777244a81b66a74631fb29148/artifacts/grid_search__6',
    'mlruns/721665528993214933/3aa73043032f48e086154fc53eb7859a/artifacts/grid_search__7',
    'mlruns/721665528993214933/853a241fffc64c8abe9ab97875bfda0f/artifacts/grid_search__8',
    'mlruns/721665528993214933/05000b3ef3d740389c5232617f8066cf/artifacts/grid_search__9',
    'mlruns/721665528993214933/39f6dcec9dfd47478762d54d65122ee0/artifacts/grid_search__10',
        ]

test_reg(model_paths,X_test,y_test)





================================| model 1 |==========================================
mae = 0.4597936183445393
mse = 0.34117934067680444
rmse = 0.5841055903488721
r2 = 0.9921098283965725


================================| model 2 |==========================================
mae = 0.46772774691058444
mse = 0.3430814356062143
rmse = 0.5857315388522409
r2 = 0.9920658402249286


================================| model 3 |==========================================
mae = 0.44924880327363587
mse = 0.3621596240099895
rmse = 0.6017969956804283
r2 = 0.9916246347870797


================================| model 4 |==========================================
mae = 0.8012375636239614
mse = 0.8703584029017738
rmse = 0.9329300096479767
r2 = 0.9798719431787478


================================| model 5 |==========================================
mae = 0.8263515801508974
mse = 0.9681927928832614
rmse = 0.9839678820384644
r2 = 0.9776094083953131


================================| model 6 |============

In [57]:
model1 = mlflow.sklearn.load_model('mlruns/721665528993214933/0a9bb5706ecf4d869ffdb50cf85781cb/artifacts/grid_search__1')
model2=mlflow.sklearn.load_model('mlruns/721665528993214933/a2a1fba817a940bba070b0bd5ddb679a/artifacts/grid_search__2')
model3=mlflow.sklearn.load_model('mlruns/721665528993214933/39f6dcec9dfd47478762d54d65122ee0/artifacts/grid_search__10')


In [58]:
from sklearn.ensemble import VotingRegressor

voting_pipe = Pipeline([
    ('voting', VotingRegressor(estimators=[
        ('grid_search20', model1.best_estimator_),
        ('grid_search22', model2.best_estimator_),
         # Model terbaik dari grid search pertama
        ('grid_search18',model3.best_estimator_)   # Model terbaik dari grid search kedua
    ]))
])
voting_pipe.fit(X_train,y_train)


Pipeline(steps=[('voting',
                 VotingRegressor(estimators=[('grid_search20',
                                              Pipeline(steps=[('StandardScaler',
                                                               StandardScaler()),
                                                              ('SVR',
                                                               SVR(C=1,
                                                                   coef0=10,
                                                                   epsilon=0.01,
                                                                   kernel='poly'))])),
                                             ('grid_search22',
                                              Pipeline(steps=[('StandardScaler',
                                                               StandardScaler()),
                                                              ('SVR',
                                                               SVR(C=1,
                                                                   coef0=10,
                                                                   epsilon=0.001,
                                                                   kernel='poly'))])),
                                             ('grid_search18',
                                              Pipeline(steps=[('StandardScaler',
                                                               StandardScaler()),
                                                              ('SVR',
                                                               SVR(C=0.001,
                                                                   coef0=10,
                                                                   degree=7,
                                                                   kernel='poly'))]))]))])

In [59]:
mod18=model1.predict(X_test)
mod20=model2.predict(X_test)
mod22=model3.predict(X_test)
vote=voting_pipe.predict(X_test)


In [60]:
y_test=y_test.reset_index(drop=True)

In [61]:
gabungan=pd.concat([y_test,pd.DataFrame(mod18),pd.DataFrame(mod20),pd.DataFrame(mod22),pd.DataFrame(vote)],axis=1)
gabungan.columns=(['Actual','Predicted 1','Predicted 3','Predicted 2','vote'])
gabungan=gabungan.sort_values(['Actual'])
gabungan=gabungan.reset_index(drop=True)
gabungan

,Actual,Predicted 1,Predicted 3,Predicted 2,vote
0,26.28,26.139539,26.108935,25.753526,26.000667
1,28.29,28.301203,28.291347,28.058882,28.217144
2,30.97,30.521076,30.502130,30.738453,30.587220
3,34.99,35.047677,35.068603,35.022275,35.046185
4,35.52,35.897330,35.913334,35.644618,35.818428
5,36.59,36.902153,36.921160,35.882744,36.568686
6,41.14,40.252419,40.257330,40.015864,40.175204
7,44.08,42.761336,42.774667,42.698993,42.744998
8,44.08,44.561470,44.567166,44.122779,44.417138
9,44.48,45.052118,45.056602,44.608824,44.905848


In [69]:
gabungan.to_csv('data/data_csv/hasil1.csv', index=False)

In [65]:



import plotly.graph_objs as go
x=gabungan.index.values
# Define traces for each line
trace_actual = go.Scatter(x=x, y=gabungan['Actual'].values, mode='lines', name='Actual',line=dict(color='red'))
trace_pred_1 = go.Scatter(x=x, y=gabungan['Predicted 1'].values, mode='lines', name='Predicted 1',line=dict(color='blue'))
trace_pred_2 = go.Scatter(x=x, y=gabungan['Predicted 2'].values, mode='lines', name='Predicted 2',line=dict(color='yellow'))
trace_pred_3 = go.Scatter(x=x, y=gabungan['Predicted 3'].values, mode='lines', name='Predicted 3',line=dict(color='green'))
trace_pred_v = go.Scatter(x=x, y=gabungan['vote'].values, mode='lines', name='Vote',line=dict(color='green'))

# Create the plotly figure
fig = go.Figure()

# Add traces to the figure
fig.add_trace(trace_actual)
fig.add_trace(trace_pred_1)
fig.add_trace(trace_pred_2)
fig.add_trace(trace_pred_3)
fig.add_trace(trace_pred_v)

# Update layout
fig.update_layout(
    title="Comparison of Actual and Predicted Values",
    xaxis_title="X Axis Title",
    yaxis_title="Y Axis Title",
    #xaxis_tickfont=dict(size=10)
)

# Show plot
fig.show()

In [71]:
import joblib

# Simpan pipeline ke dalam file menggunakan joblib
joblib.dump(voting_pipe, "data/data_model/voting_pipeline.joblib")


['data/data_model/voting_pipeline.joblib']